In [1]:
import pandas as pd

RP-Mod & RP-Crowd

In [2]:
filename = 'RP-Mod-Crowd'
df_raw = pd.read_csv(f'../ExternalData/{filename}.csv', sep=',')

In [3]:
df_raw.head(2)

,Unnamed: 0.1,Unnamed: 0,id,Text,Reject Newspaper,Reject Crowd,Rejection Count Crowd,Sexism Count Crowd,Racism Count Crowd,Threat Count Crowd,Insult Count Crowd,Profanity Count Crowd,Meta Count Crowd,Advertisement Count Crowd
0,0,0,1911223,Niemand braucht Laschet den Merkel Günstling !...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1911225,das war apokalypse now. nicht einmal zu einem ...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Cross table of the columns Reject Crowd and Reject Newspaper
pd.crosstab(df_raw['Reject Crowd'], df_raw['Reject Newspaper'])

Reject Newspaper,0,1
Reject Crowd,,
0,74859,5625
1,3000,1516


In [4]:
df_out = df_raw[['Text', 'Reject Newspaper', 'Reject Crowd']].rename(columns={'Text': 'text', 'Reject Newspaper': 'label_newspaper', 'Reject Crowd': 'label_crowd'})

In [5]:
# Create a new column named 'label' that is the OR of 'label_newspaper' and 'label_crowd'
df_out['label'] = df_out[['label_newspaper', 'label_crowd']].max(axis=1)

In [6]:
df_out['text'] = df_out['text'].str.replace(',','')


In [7]:
df_out.head(3)

,text,label_newspaper,label_crowd,label
0,Niemand braucht Laschet den Merkel Günstling !...,0,0,0
1,das war apokalypse now. nicht einmal zu einem ...,0,0,0
2,Katastrophal - Katastrophal - anders kann d...,0,0,0


In [8]:
df_out.to_csv(f'../ExternalData/{filename}_Clean.csv', sep=',', index=False)

In [9]:

n_minority = df_out.label.value_counts(ascending=True)[1]
print(n_minority)

10141


In [10]:
# Sample of the data
df_out_sub = df_out.groupby('label').apply(lambda x: x.sample(n=min(n_minority, len(x))))


In [11]:
df_out_sub.label.value_counts()

label
0    10141
1    10141
Name: count, dtype: int64

In [12]:
df_out_sub.to_csv(f'../ExternalData/{filename}_Clean_Sub.csv', sep=',', index=False)
df_out_sub.to_parquet(f'../ExternalData/{filename}_Clean_Sub.parquet', index=False)

IWG_hatespeech_public

In [50]:
test.shape

(308, 4)

In [13]:
filename = 'german_hatespeech_refugees'
df_raw = pd.read_csv(f'../ExternalData/{filename}.csv', sep=',')

In [14]:
df_raw.head(2)

,Tweet,HatespeechOrNot (Expert 1),HatespeechOrNot (Expert 2),Hatespeech Rating (Expert 2)
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,YES,NO,5
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,NO,NO,1


In [15]:
df_out = (df_raw[['Tweet', 'HatespeechOrNot (Expert 1)', 'HatespeechOrNot (Expert 2)']]
          .rename(columns={'Tweet': 'text', 'HatespeechOrNot (Expert 1)': 'label_expert1', 'HatespeechOrNot (Expert 2)': 'label_expert2'}))

In [16]:
df_out['text'] = df_out['text'].str.replace(',','')


In [17]:
df_out['label_expert1'] = df_out['label_expert1'].replace({'NO': 0, 'YES': 1})
df_out['label_expert2'] = df_out['label_expert2'].replace({'NO': 0, 'YES': 1})


In [18]:
# Create a new column with the max vote of the two "experts"
df_out['label'] = df_out[['label_expert1', 'label_expert2']].max(axis=1)

In [19]:
df_out.to_csv(f'../ExternalData/{filename}_Clean.csv', sep=',', index=False)

In [20]:
df_out.shape

(469, 4)

In [21]:
n_minority = df_out.label.value_counts(ascending=True)[1]
print(n_minority)

154


In [22]:
df_out_sub = df_out.groupby('label').apply(lambda x: x.sample(n=min(n_minority, len(x))))

In [23]:
df_out_sub.label.value_counts()

label
0    154
1    154
Name: count, dtype: int64

In [24]:
df_out_sub.to_csv(f'../ExternalData/{filename}_Clean_Sub.csv', sep=',', index=False)
df_out_sub.to_parquet(f'../ExternalData/{filename}_Clean_Sub.parquet', index=False)

Facebook Comments

In [25]:
filename_comments = 'fb_comments'
filename_annotations = 'fb_annotated_comments'
df_raw_comments = pd.read_csv(f'../ExternalData/{filename_comments}.csv', sep=',')
df_raw_annotations = pd.read_csv(f'../ExternalData/{filename_annotations}.csv', sep=',')

In [26]:
df_raw_comments.shape

(5836, 5)

In [27]:
df_raw_annotations['annotator_id'] = df_raw_annotations['annotator_id'].replace({1: 'Annotator_1', 3: 'Annotator_3'})

In [28]:
df_raw_annotations.entry_number.value_counts()

entry_number
1    1295
2      34
0       2
Name: count, dtype: int64

In [29]:
# Remove Entries with value equal to 2
df_raw_annotations = df_raw_annotations.loc[df_raw_annotations.entry_number != 2]

In [30]:
df_raw_annotations.shape

(1297, 5)

In [31]:
# Pivot df_raw_annotations to get the annotations in columns
df_raw_annotations_pivot = pd.pivot_table(df_raw_annotations, index='comment_id', columns='annotator_id', values='valence', fill_value=0)

In [32]:
df_raw_annotations_pivot.sort_values(by='Annotator_1', ascending=False)

annotator_id,Annotator_1,Annotator_3
comment_id,,
1,2,2
2547,2,2
2994,2,2
2993,2,2
2988,2,2
...,...,...
45,0,1
4006,0,1
1013,0,1


In [33]:
df_raw_annotations_pivot.max(axis=0)

annotator_id
Annotator_1    2
Annotator_3    2
dtype: int64

In [34]:
df_merged = pd.merge(df_raw_comments, df_raw_annotations_pivot, on='comment_id', how='left')

In [35]:
df_merged.shape

(5836, 7)

In [36]:
df_merged.head(5)

,comment_id,post_id,anonymized_user,message,created_at,Annotator_1,Annotator_3
0,1,1,d766dfb93914e8da,gleich an die wand stellen und erschiessen..,2015-03-28 01:59:28,2.0,2.0
1,2,1,6e49f84e2a5a1f05,nicht dass ich der Grundbotschaft dieses Post'...,2015-03-28 01:59:28,NaN,NaN
2,3,1,6ab1ef4615c45a79,"Das mit dem ""an die Wand stellen und erschiessen""",2015-03-28 01:59:28,NaN,NaN
3,4,1,6ab1ef4615c45a79,"Seit dem ""an die Wand stellen und erschiessen""...",2015-03-28 01:59:28,NaN,NaN
4,5,1,03c34eadeee054d7,Ja ja die Kriminelle Heimatpartei FPÖ von Kind...,2015-03-28 01:59:28,NaN,NaN


In [37]:
df_out = df_merged[['message', 'Annotator_1', 'Annotator_3']].fillna(0).rename(columns={'message': 'text', 'Annotator_1': 'label_annotator1', 'Annotator_3': 'label_annotator3'})

In [38]:
df_out['label_annotator1'] = df_out['label_annotator1'].apply(lambda x: 1 if x > 0 else 0)
df_out['label_annotator3'] = df_out['label_annotator3'].apply(lambda x: 1 if x > 0 else 0)

In [39]:
df_out['label'] = df_out[['label_annotator1', 'label_annotator3']].max(axis=1)   

In [40]:
df_out['text'] = df_out['text'].str.replace(',','')


In [41]:
df_out.head(20)

,text,label_annotator1,label_annotator3,label
0,gleich an die wand stellen und erschiessen..,1,1,1
1,nicht dass ich der Grundbotschaft dieses Post'...,0,0,0
2,"Das mit dem ""an die Wand stellen und erschiessen""",0,0,0
3,"Seit dem ""an die Wand stellen und erschiessen""...",0,0,0
4,Ja ja die Kriminelle Heimatpartei FPÖ von Kind...,0,0,0
5,Ihr seids empfindlich . . Is beste wos uns pas...,1,1,1
6,Mei bitte sie sollen sich alle gegenseitig abs...,1,1,1
7,Wahnsinn.... :-(,0,0,0
8,Alles nur traumatisierte Kindlein. Die von de...,0,0,0
9,Fachkräfte bei der Arbeit. Ein Hirnchirurg und...,0,0,0


In [42]:
df_out.to_csv(f'../ExternalData/fb_Clean.csv', sep=',', index=False)

In [43]:
n_minority = df_out.label.value_counts(ascending=True)[1]
print(n_minority)

674


In [44]:
df_out_sub = df_out.groupby('label').apply(lambda x: x.sample(n=min(n_minority, len(x))))

In [45]:
df_out_sub.label.value_counts()

label
0    674
1    674
Name: count, dtype: int64

In [46]:
df_out_sub.to_csv(f'../ExternalData/fb_Clean_Sub.csv', sep=',', index=False)
df_out_sub.to_parquet(f'../ExternalData/fb_Clean_Sub.parquet', index=False)

In [48]:
any(df_out_sub.text.str.contains(','))

False